# __Follwing Social Network Creation__

In [1]:
import pandas as pd
import datetime, os, tweepy
from credentials import *

### Getting the nodes (users) from all days

In [2]:
DATA_PATH_TT = 'data/tweets/'
DATA_PATH_NT = 'data/networks/'

In [3]:
week_list = [week_dir for week_dir in os.listdir(DATA_PATH_NT) if os.path.isdir(DATA_PATH_TT+week_dir) and not week_dir.endswith('.ipynb_checkpoints')]

week_list.sort()
week_list.remove('week_pr_01')
week_list.insert(week_list.index('week_12')+1, 'week_pr_01')
week_list.remove('week_pr_02')
week_list.insert(week_list.index('week_pr_01')+1, 'week_pr_02')
week_list.remove('week_pr_03')
week_list.insert(week_list.index('week_17')+1, 'week_pr_03')

print(week_list)

['week_01', 'week_02', 'week_03', 'week_04', 'week_05', 'week_06', 'week_07', 'week_08', 'week_09', 'week_10', 'week_11', 'week_12', 'week_pr_01', 'week_pr_02', 'week_13', 'week_14', 'week_15', 'week_16', 'week_17', 'week_pr_03', 'week_18', 'week_19', 'week_20', 'week_21']


In [4]:
day_list = ['day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7']
node_list = []

for week in week_list:
    print(f'Starting week {week}...\n')
    for day in day_list:
        print(day+'...')
        nodes = pd.read_csv(DATA_PATH_NT+week+'/'+day+'/'+'gephi_nodes.csv')
        for usr, disp_n, desc in nodes[['Id', 'DisplayName', 'Description']].values:
            if (usr, disp_n, desc) not in node_list:
                node_list.append((usr, disp_n, desc))

all_nodes = pd.DataFrame(node_list, columns=['Id', 'DisplayName', 'Description'])
all_nodes.to_csv(DATA_PATH_NT+'all_nodes.csv', index=False)

Starting week week_01...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_02...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_03...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_04...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_05...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_06...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_07...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_08...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_09...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_10...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_11...

day_1...
day_2...
day_3...
day_4...
day_5...
day_6...
day_7...
Starting week week_12

In [7]:
all_nodes.drop_duplicates(subset='Id', keep='last', inplace=True)
all_nodes.shape

(536263, 3)

In [8]:
all_nodes.to_csv(DATA_PATH_NT+'all_nodes.csv', index=False)

### Following Social Network Edges

In [109]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

TypeError: __init__() got an unexpected keyword argument 'wait_on_rate_limit_notify'

In [62]:
follows_list = []
susp_account = []

for usr in all_nodes['Id'].values.tolist():
    try:
        usr_protected = api.get_user(screen_name=usr).protected
        if not usr_protected:
            for usr2 in all_nodes['Id']:
                if usr != usr2:
                    friendship = api.get_friendship(source_screen_name=usr, target_screen_name=usr2)
                    if friendship[0].following:
                        follows_list.append((usr, usr2))
    except tweepy.errors.TweepyException as e:
        susp_account.append(usr)

Rate limit reached. Sleeping for: 333
Rate limit reached. Sleeping for: 782
Rate limit reached. Sleeping for: 781
Rate limit reached. Sleeping for: 776
Rate limit reached. Sleeping for: 774
Rate limit reached. Sleeping for: 778
Rate limit reached. Sleeping for: 780
Rate limit reached. Sleeping for: 784
Rate limit reached. Sleeping for: 779
Rate limit reached. Sleeping for: 783
Rate limit reached. Sleeping for: 783
Rate limit reached. Sleeping for: 784
Rate limit reached. Sleeping for: 779
Rate limit reached. Sleeping for: 775
Rate limit reached. Sleeping for: 769
Rate limit reached. Sleeping for: 775
Rate limit reached. Sleeping for: 781
Rate limit reached. Sleeping for: 774
Rate limit reached. Sleeping for: 776
Rate limit reached. Sleeping for: 772
Rate limit reached. Sleeping for: 771
Rate limit reached. Sleeping for: 759
Rate limit reached. Sleeping for: 774
Rate limit reached. Sleeping for: 753
Rate limit reached. Sleeping for: 770
Rate limit reached. Sleeping for: 760
Rate limit r

KeyboardInterrupt: 

91 nós, 123 arestas

In [107]:
friends = api.get_friends(screen_name='lucasRniere')

In [108]:
for user in friends:
    print(user._json['screen_name'])

NASAHubble
SciencePorn1
lucasinutilismo
hermeserenatooc
NerdBoomer
diogofpacheco
randolfeap
camarotedacpi
demori
JeanLafleur
noseconexoes
MarceloFreixo
kdnuggets
arcelinobsilva
liabolica
VeroInstituto
dadourado
OSoMe_IU
Botometer
iqciencia
